In [2]:
import os

In [3]:
def make_qchem_input_mixed(fl, path2efp, path2qmefp, path2gro):
    
    """Make qchem input files
    Inputs: - fl         : LibEFP output file
            - path2efp   : path to LibEFP output files
            - path2qmefp : path to folder where QM/EFP inputs will be written 
            - path2gro   : path to folder with gro files (the ones used to create LibEFP inputs)
    """
    
    qmefp_details = [
    '   METHOD               wB97X\n',
    '   BASIS                6-31G*\n',
    '   JOBTYPE              sp\n',
    '   EFP_COORD_XYZ        true\n',
    '   EFP_FRAGMENTS_ONLY   false\n',
    '   EFP_DISP_DAMP        1\n',
    '   EFP_QM_DISP          true\n',
    '   EFP_QM_EXREP         true\n',
    '   EFP_DISP_DAMP        2\n',
    '   SYM_IGNORE           true\n',
    ]

    molecules_dict = {'ammonia' : 4, 'methane' : 5, 'methanol' : 6, 'water' : 3}
    name = fl.split('.')[0]
    
    if name in molecules_dict.keys():
        efp_name = name.split('_')[1]
        qm_atoms = molecules_dict[name.split('_')[0]]
        solv_atoms = molecules_dict[name.split('_')[1]]
    else:
        mobley_name = 'mobley_'+name.split('_')[1]+'.gro'
        other_name = name.split('_')[0] + '_centered_solv_' + name.split('_')[1] + '.gro'
        gro_name = mobley_name if 'mobley' in name else other_name
        efp_name = name.split('_')[3]
        solv_atoms = molecules_dict[efp_name]
        with open(f'{path2gro}/{gro_name}') as gro:
            gro_lns = gro.readlines()
        qm_atoms = len([ln for ln in gro_lns if '1MOL' in ln or '1UNK' in ln])

    with open(f'{path2efp}/{fl}', 'r') as ef:
        ef_lns = ef.readlines()
    if 'JOB COMPLETED SUCCESSFULLY' in ef_lns[-3]:
        geo_iidx = ef_lns.index('    GEOMETRY (ANGSTROMS)\n')+2
        geo_fidx = ef_lns.index('    ENERGY COMPONENTS (ATOMIC UNITS)\n')-2
        qm_coords = ef_lns[geo_iidx:geo_iidx+qm_atoms]
        efp_coords = ef_lns[geo_iidx+qm_atoms:geo_fidx]
        len_efp = list(range(0, len(efp_coords)))
        efp_idx = len_efp[::solv_atoms]+len_efp[1::solv_atoms]+len_efp[2::solv_atoms]
        efp_idx.sort()
        efp_coords = [efp_coords[c] for c in efp_idx]
        qmefp_input = open(f'{path2qmefp}/{name}_qmefp.in', 'w+')
        qmefp_input.write('$molecule\n   0 1\n')
        for line in qm_coords:
            atom = line.split()[0][3]
            xyz = line[6:]
            qmefp_input.write(f'   {atom} {xyz}')
        qmefp_input.write('$end\n\n$rem\n')
        for line in qmefp_details:
            qmefp_input.write(line)
        qmefp_input.write('$end\n\n$efp_fragments\n')
        qmefp_input.write(f'{efp_name}\n')
        for line in efp_coords[0:-1]:
            if line[0:3] == 'A03':
                qmefp_input.write(f'   {line}')
                qmefp_input.write(f'{efp_name}\n')
            else:
                qmefp_input.write(f'   {line}')
        qmefp_input.write(f'   {efp_coords[-1]}')
        qmefp_input.write('$end\n\n')
        qmefp_input.close()

    return('done!')

In [4]:
main_path = os.getcwd().replace('D_MakeQChem', '')
path2efp = f'{main_path}/C_MakeLibEFP'
path2qmefp = os.getcwd()
path2gro = path2efp
fls = [fl for fl in os.listdir(path2efp) if fl.endswith('.out') and 'slurm' not in fl]
for fl in fls:
    make_qchem_input_mixed(fl, path2efp, path2qmefp, path2gro)